In [1]:
import pandas as pd
import numpy as np

In [2]:
file_path = r"C:\Users\adity\Downloads\Electric_Vehicle_Registration_Data.csv"
df = pd.read_csv(file_path)

In [3]:
# 1. Description of the dataset
print("Dataset Info:\n")
df.info()
print("\nDataset Description:\n")
print(df.describe())

Dataset Info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52691 entries, 0 to 52690
Data columns (total 20 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   ID                             52691 non-null  int64 
 1   Plate Type                     52691 non-null  object
 2   Primary Customer City          52657 non-null  object
 3   Primary Customer State         52657 non-null  object
 4   Registration Start Date        52691 non-null  object
 5   Registration Expiration Date   52691 non-null  object
 6   Registration Usage             52691 non-null  object
 7   Vehicle Type                   52691 non-null  object
 8   Vehicle Weight                 52691 non-null  int64 
 9   Vehicle Year                   52691 non-null  int64 
 10  Vehicle Make                   52691 non-null  object
 11  Vehicle Model                  52691 non-null  object
 12  Vehicle Body                   52691 non-null

Observation: Total columns = 20 with misisng data in Primary Customer City and Primary Customer State columns

In [4]:
# Drop non-useful columns
columns_to_drop = ['ID', 'Vehicle Name']
df.drop(columns=columns_to_drop, axis=1, inplace=True)

print("Updated Dataset Columns:\n", df.columns)


Updated Dataset Columns:
 Index(['Plate Type', 'Primary Customer City', 'Primary Customer State',
       'Registration Start Date', 'Registration Expiration Date',
       'Registration Usage', 'Vehicle Type', 'Vehicle Weight', 'Vehicle Year',
       'Vehicle Make', 'Vehicle Model', 'Vehicle Body', 'Primary Color',
       'Vehicle Declared Gross Weight', 'Fuel Code', 'Vehicle Recorded GVWR',
       'Type', 'Vehicle Category'],
      dtype='object')


Since ID is unique identifier it wont be useful for insights and vehicle name is redundant since we have vehicle make and vehicle model present

In [5]:
# Drop rows with maximum missing values (allowing only one missing column)
df.dropna(thresh=df.shape[1] - 1, inplace=True)

print(f"Dataset shape after dropping rows: {df.shape}")
print("Remaining missing values per column:\n", df.isnull().sum())


Dataset shape after dropping rows: (52657, 18)
Remaining missing values per column:
 Plate Type                       0
Primary Customer City            0
Primary Customer State           0
Registration Start Date          0
Registration Expiration Date     0
Registration Usage               0
Vehicle Type                     0
Vehicle Weight                   0
Vehicle Year                     0
Vehicle Make                     0
Vehicle Model                    0
Vehicle Body                     0
Primary Color                    0
Vehicle Declared Gross Weight    0
Fuel Code                        0
Vehicle Recorded GVWR            0
Type                             0
Vehicle Category                 0
dtype: int64


Now handling missing data :-
1. Numeric Columns are replaced with mean of the coulmn
2. Categorical Columns are replaced by mode

In [7]:
# Fill missing values only for numeric columns with the mean
df.fillna(df.select_dtypes(include=['number']).mean(), inplace=True)

# Fill missing values for categorical columns with the mode
for col in df.select_dtypes(include='object').columns:
    df[col].fillna(df[col].mode()[0], inplace=True)

# Verify no missing values remain
print("Remaining missing values per column after filling:\n", df.isnull().sum())


Remaining missing values per column after filling:
 Plate Type                       0
Primary Customer City            0
Primary Customer State           0
Registration Start Date          0
Registration Expiration Date     0
Registration Usage               0
Vehicle Type                     0
Vehicle Weight                   0
Vehicle Year                     0
Vehicle Make                     0
Vehicle Model                    0
Vehicle Body                     0
Primary Color                    0
Vehicle Declared Gross Weight    0
Fuel Code                        0
Vehicle Recorded GVWR            0
Type                             0
Vehicle Category                 0
dtype: int64


Dummy variables help encode categorical data into numerical values suitable for machine learning models. We use pd.get_dummies() to do so while avoiding dummy variable trap by dropping one level per category.

In [8]:
# Identify categorical columns
categorical_cols = df.select_dtypes(include='object').columns

# Create dummy variables
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

print(f"Dataset shape after creating dummy variables: {df.shape}")
print("First few rows:\n", df.head())


Dataset shape after creating dummy variables: (52657, 3364)
First few rows:
    Vehicle Weight  Vehicle Year  Vehicle Declared Gross Weight  \
0               0          2021                              0   
1               0          2024                              0   
2               0          2024                              0   
3               0          2024                              0   
4               0          2021                              0   

   Vehicle Recorded GVWR  Plate Type_Combination  Plate Type_Commercial  \
0                      0                   False                  False   
1                      0                   False                  False   
2                      0                   False                  False   
3                      0                   False                  False   
4                      0                   False                  False   

   Plate Type_Factory  Plate Type_Farm  Plate Type_Handicapped  \
0        

We can identify outliers using the Interquartile Range (IQR) method, which is a common approach. The IQR is calculated as the difference between the 75th percentile (Q3) and the 25th percentile (Q1). Outliers are typically defined as data points that fall outside the range [Q1−1.5×IQR,Q3+1.5×IQR].

In [9]:
# Function to detect outliers using IQR
def detect_outliers(df):
    outliers = {}
    for col in df.select_dtypes(include=['number']).columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers[col] = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    return outliers

# Get outliers
outliers = detect_outliers(df)

# Show outliers for each column
for col, outlier_data in outliers.items():
    print(f"Outliers in column '{col}':\n", outlier_data)



Outliers in column 'Vehicle Weight':
        Vehicle Weight  Vehicle Year  Vehicle Declared Gross Weight  \
14               4800          2021                              0   
18               5600          2021                              0   
27               3488          2021                              0   
49              10360          2019                              0   
51               4800          2021                              0   
...               ...           ...                            ...   
52659            9500          2023                              0   
52670            8250          2023                        1249925   
52674            9990          2024                              0   
52683            9500          2022                              0   
52684            5300          2024                        6143108   

       Vehicle Recorded GVWR  Plate Type_Combination  Plate Type_Commercial  \
14                      5300              

Standardization (Z-score scaling) involves subtracting the mean and dividing by the standard deviation. This results in a distribution with a mean of 0 and a standard deviation of 1. It's typically used for algorithms like SVM, k-means, and PCA.

Normalization (Min-Max scaling) rescales the data to a fixed range, usually [0, 1]. This is useful for algorithms like neural networks and k-nearest neighbors.

In [10]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Standardize numeric columns
scaler = StandardScaler()
df_standardized = df.copy()
df_standardized[df.select_dtypes(include=['number']).columns] = scaler.fit_transform(df[df.select_dtypes(include=['number']).columns])

# Normalize numeric columns
normalizer = MinMaxScaler()
df_normalized = df.copy()
df_normalized[df.select_dtypes(include=['number']).columns] = normalizer.fit_transform(df[df.select_dtypes(include=['number']).columns])

# Show first few rows of standardized and normalized data
print("Standardized Data (first few rows):\n", df_standardized.head())
print("Normalized Data (first few rows):\n", df_normalized.head())


Standardized Data (first few rows):
    Vehicle Weight  Vehicle Year  Vehicle Declared Gross Weight  \
0       -0.311374      -0.25558                      -0.387129   
1       -0.311374       0.95799                      -0.387129   
2       -0.311374       0.95799                      -0.387129   
3       -0.311374       0.95799                      -0.387129   
4       -0.311374      -0.25558                      -0.387129   

   Vehicle Recorded GVWR  Plate Type_Combination  Plate Type_Commercial  \
0              -0.350774                   False                  False   
1              -0.350774                   False                  False   
2              -0.350774                   False                  False   
3              -0.350774                   False                  False   
4              -0.350774                   False                  False   

   Plate Type_Factory  Plate Type_Farm  Plate Type_Handicapped  \
0               False            False           

Save the dataset to new CSV file

In [ ]:
# Save the standardized dataset to CSV
df_standardized.to_csv(r'C:\Users\adity\Downloads\Electric_Vehicle_Registration_Standardized.csv', index=False)

# Save the normalized dataset to CSV
df_normalized.to_csv(r'C:\Users\adity\Downloads\Electric_Vehicle_Registration_Normalized.csv', index=False)

print("Standardized and Normalized datasets saved successfully.")
